In [6]:
#begin by importing your lbiraries 
import pandas as pd
import numpy as np 
import csv
import requests
from bs4 import BeautifulSoup  #Beautifulsoup is used to separate out the HTML
get_ipython().system('conda install beautifulsoup4')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h7b6447c_4         3.7 MB
    ca-certificates-2020.1.1   |                0         132 KB
    beautifulsoup4-4.8.2       |           py36_0         161 KB
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.2-py36_0     
    ca-certificates: 2019.11.27-0      --> 2020.1.1-0       
    openssl:         1.1.1d-h7b6447c_3 --> 1.1.1d-h7b6447c_4


openssl-1.1.1d       | 3.7 MB    | ##################################### | 100% 
ca-certificates-2020 | 132 KB    | ##################################### | 10

In [7]:
pd.set_option('max_colwidth', 800) 


# Preprocessing and table extraction from Wikipedia

In [15]:
#This is a request to pull the wikipedia page in. 
Page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 


In [16]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text
#using the soup library
soup = BeautifulSoup(raw_wikipedia_page,'xml')



In [20]:
# Creating our Table
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []



#Make sure our table is clean when we pull it from wikipedia
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
        counter +=1
  
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        Postcode.append(Postcode_var)
        Borough.append(Borough_var)
        Neighbourhood.append(Neighbourhood_var)

In [21]:
#Organizing multiple neighborhood postal codes 
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


# Identifying unique postal codes from the table

In [22]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M8W,Etobicoke,"Alderwood, Long Branch"
1,M5S,Downtown Toronto,University of Toronto
2,M8V,Etobicoke,New Toronto
3,M2H,North York,Hillcrest Village
4,M4Y,Downtown Toronto,Church and Wellesley


In [13]:
df_toronto.shape


(1, 3)

# Part Two: Bringing in the lat, long

In [14]:
#Install folium and import library for our next section
!pip install folium

In [42]:
import folium
import json, lxml
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [23]:
df = pd.read_csv (r'http://cocl.us/Geospatial_data')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
#We rename the column name for toronto to match the other table
df_toronto.rename(columns = {"Postcode": "Postal Code"}, inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M8W,Etobicoke,"Alderwood, Long Branch"
1,M5S,Downtown Toronto,University of Toronto
2,M8V,Etobicoke,New Toronto
3,M2H,North York,Hillcrest Village
4,M4Y,Downtown Toronto,Church and Wellesley


In [26]:
#Merge the tables on postal code and create a new dataframe 
canada_df = pd.merge(df_toronto, df, on = "Postal Code")
canada_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
1,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049
2,M8V,Etobicoke,New Toronto,43.605647,-79.501321
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
5,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
6,M2L,North York,York Mills,43.757490,-79.374714
7,M4T,Central Toronto,Moore Park,43.689574,-79.383160
8,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
9,M6M,York,"Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013


# Part 3: Clustering Toronto Neighborhoods


In [27]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#REMEMBER THAT CAPITAL LETTERS ARE DIFFERENT THAN lower case letters.. It is important 
# add markers to map
for lat, lng, borough, neighbourhood in zip(canada_df['Latitude'], canada_df['Longitude'], canada_df['Borough'], canada_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Connecting to Foursquare 

In [29]:
{
    "tags": [
        "hide_input",
    ]
}
CLIENT_ID = '1TQX3BIGE0AVURRWD1YKVRLYVZZA5A3MQ52OZOEDFZBKVMJW' # your Foursquare ID
CLIENT_SECRET = 'E513XDA24031SHHX4EBAU4G2PY0CUYXAASQZKNHNZ4LRX1XQ' # your Foursquare Secret
VERSION = '20200215' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1TQX3BIGE0AVURRWD1YKVRLYVZZA5A3MQ52OZOEDFZBKVMJW
CLIENT_SECRET:E513XDA24031SHHX4EBAU4G2PY0CUYXAASQZKNHNZ4LRX1XQ


In [36]:
canada_df.loc[0, 'Neighbourhood']

'Alderwood, Long Branch'

In [34]:
neighbourhood_latitude = canada_df.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = canada_df.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = canada_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Alderwood, Long Branch are 43.60241370000001, -79.54348409999999.


In [37]:
# type your answer here
LIMIT = 100
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1TQX3BIGE0AVURRWD1YKVRLYVZZA5A3MQ52OZOEDFZBKVMJW&client_secret=E513XDA24031SHHX4EBAU4G2PY0CUYXAASQZKNHNZ4LRX1XQ&v=20200215&ll=43.60241370000001,-79.54348409999999&radius=500&limit=100'

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e486473882fc7001bb9df09'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Alderwood',
  'headerFullLocation': 'Alderwood, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.60691370450001,
    'lng': -79.53728145264961},
   'sw': {'lat': 43.597913695500004, 'lng': -79.54968674735036}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c47927c1ddec928fbec9d32',
       'name': 'Il Paesano Pizzeria & Restaurant',
       'location': {'address': '396 Browns Line',
        'crossStreet': 'at Horner Ave',
        'lat': 43.60128,
        'lng': -79.545028,
        'labeledLatLngs': [{'l

In [39]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Il Paesano Pizzeria & Restaurant,Pizza Place,43.601280,-79.545028
1,Toronto Gymnastics International,Gym,43.599832,-79.542924
2,Timothy's Pub,Pub,43.600165,-79.544699
3,Tim Hortons,Coffee Shop,43.602396,-79.545048
4,Pizza Pizza,Pizza Place,43.605340,-79.547252


# Next we will segment the neighbourhoods based on their most popular venue type

In [59]:

def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]['categories'][0]["name"]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Neighbourhood", 
                  "Neighbourhood Latitude", 
                  "Neighbourhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Category"]
    
    return(nearby_venues)

In [60]:
#New Data frame of the toronto venues
toronto_venues = getNearbyVenues(names=canada_df["Neighbourhood"],
                                   latitudes=canada_df["Latitude"],
                                   longitudes=canada_df["Longitude"]
                                  )

Alderwood, Long Branch
University of Toronto
New Toronto
Hillcrest Village
Church and Wellesley
Cliffcrest, Cliffside
York Mills
Moore Park
Scarborough Village
Keelesdale, Mount Dennis, Silverthorn
Mimico NW, The Queensway West
Dovercourt Village
Bayview Village
Chinatown, Grange Park, Kensington Market
Queen's Park
Weston
Runnymede, Swansea
Emery, Humberlea
Tam O'Shanter
Humber Bay, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
CFB Toronto
St. James Town
Lawrence Heights, Lawrence Manor
Leaside
East Toronto
Kingsview Village
Northwood Park, York University
Upper Rouge
Victoria Village
The Beaches
First Canadian Place, Underground city
Deer Park, Rathnelly, South Hill
Downsview West
Flemingdon Park
Willowdale West
Parkdale, Roncesvalles
Harbourfront
Toronto Islands, Union Station
Birch Cliff
Agincourt North, Milliken
Bathurst Manor, Wilson Heights
Markland Wood
Commerce Court
Rouge, Malvern
Woburn
Little Portugal, Trinity
Berczy Park
Design Exchange, To

In [62]:
print(toronto_venues.shape)
toronto_venues.head()

(983, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Alderwood, Long Branch",43.602414,-79.543484,Il Paesano Pizzeria & Restaurant,43.601280,-79.545028,Pizza Place
1,"Alderwood, Long Branch",43.602414,-79.543484,Toronto Gymnastics International,43.599832,-79.542924,Gym
2,"Alderwood, Long Branch",43.602414,-79.543484,Timothy's Pub,43.600165,-79.544699,Pub
3,"Alderwood, Long Branch",43.602414,-79.543484,Tim Hortons,43.602396,-79.545048,Coffee Shop
4,"Alderwood, Long Branch",43.602414,-79.543484,Pizza Pizza,43.605340,-79.547252,Pizza Place


In [64]:
toronto_venues.groupby("Neighbourhood").count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North, Milliken",2,2,2,2,2,2
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Beaumond Heights, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
Bedford Park,25,25,25,25,25,25
Berczy Park,30,30,30,30,30,30
Birch Cliff,4,4,4,4,4,4


# Deeper Analysis On Neighbourhoods

In [97]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio
0,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
#Be sure that neighbourhood is spelt with a u (the canadian way) 
toronto_grouped = toronto_onehot.groupby("Neighbourhood").mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(75, 209)


,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Agincourt North, Milliken",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bathurst Manor, Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
toronto_grouped = toronto_grouped.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, Milliken",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Bathurst Manor, Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Beaumond Heights, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bedford Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,Birch Cliff,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,CFB Toronto,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [103]:
#Confirm the shape of the table
toronto_grouped.shape

(75, 209)

In [104]:
#Print each neighbourhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped["Neighbourhood"]:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped["Neighbourhood"] == hood].T.reset_index()
    temp.columns = ["venue","freq"]
    temp = temp.iloc[1:]
    temp["freq"] = temp["freq"].astype(float)
    temp = temp.round({"freq": 2})
    print(temp.sort_values("freq", ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1  Latin American Restaurant   0.2
2             Clothing Store   0.2
3               Skating Rink   0.2
4             Breakfast Spot   0.2


----Agincourt North, Milliken----
                venue  freq
0          Playground   0.5
1                Park   0.5
2               Motel   0.0
3  Mac & Cheese Joint   0.0
4              Market   0.0


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2      Pharmacy  0.11
3   Coffee Shop  0.11
4          Pool  0.11


----Bathurst Manor, Wilson Heights----
           venue  freq
0    Coffee Shop  0.11
1       Pharmacy  0.06
2           Bank  0.06
3  Shopping Mall  0.06
4    Bridal Shop  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Beaumond Heights, 

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [106]:
# top 10 nearby venue

import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Columns according to how many new venues 
columns = ["Neighbourhood"]
for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1, indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))

# New DF
toronto_venues_sorted = pd.DataFrame(columns = columns)
toronto_venues_sorted["Neighbourhood"] = toronto_grouped["Neighbourhood"]

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Clothing Store,Skating Rink,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,"Agincourt North, Milliken",Playground,Park,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
2,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pool,Pharmacy,Gym,Skating Rink,Pub,Sandwich Place,Department Store,Dessert Shop
3,"Bathurst Manor, Wilson Heights",Coffee Shop,Bank,Pizza Place,Middle Eastern Restaurant,Deli / Bodega,Restaurant,Diner,Sandwich Place,Bridal Shop,Shopping Mall
4,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Dim Sum Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
5,"Beaumond Heights, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",Grocery Store,Fried Chicken Joint,Pizza Place,Coffee Shop,Sandwich Place,Liquor Store,Fast Food Restaurant,Beer Store,Pharmacy,Gaming Cafe
6,Bedford Park,Coffee Shop,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Butcher,Pub,Sushi Restaurant,Indian Restaurant
7,Berczy Park,Cocktail Bar,Coffee Shop,Beer Bar,Café,Bakery,Farmers Market,Breakfast Spot,Fish Market,Basketball Stadium,Bistro
8,Birch Cliff,College Stadium,General Entertainment,Skating Rink,Café,Construction & Landscaping,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Comfort Food Restaurant
9,CFB Toronto,Airport,Park,Snack Place,Yoga Studio,Dim Sum Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [90]:
#Most Common Nearby Venues
n=5
toronto_venues_sorted['1st Most Common Venue'].value_counts()[:n].index.tolist()

['Coffee Shop', 'Café', 'Pizza Place', 'Park', 'Grocery Store']

# Cluster the Neighbourhoods

In [107]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [110]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = canada_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,1.0,Pizza Place,Coffee Shop,Pool,Pharmacy,Gym,Skating Rink,Pub,Sandwich Place,Department Store,Dessert Shop
1,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049,1.0,Café,Restaurant,Bookstore,Japanese Restaurant,Bar,Bakery,Chinese Restaurant,French Restaurant,Italian Restaurant,Beer Bar
2,M8V,Etobicoke,New Toronto,43.605647,-79.501321,1.0,Pharmacy,Fried Chicken Joint,Seafood Restaurant,Fast Food Restaurant,Restaurant,Café,Mexican Restaurant,Hobby Shop,Bakery,Liquor Store
3,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1.0,Gay Bar,Ramen Restaurant,Bookstore,Smoke Shop,Ethiopian Restaurant,Bubble Tea Shop,Burger Joint,Salon / Barbershop,Burrito Place,Juice Bar


# Map and Cluster Breakdown

In [142]:
#Create a map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[0-4],
        fill=True,
        fill_color=rainbow[0-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,0.0,Cafeteria,Park,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
7,Central Toronto,0.0,Tennis Court,Park,Yoga Studio,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
8,Scarborough,0.0,Playground,Yoga Studio,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
20,North York,0.0,Airport,Park,Snack Place,Yoga Studio,Dim Sum Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
39,Scarborough,0.0,Playground,Park,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
63,Etobicoke,0.0,River,Park,Pool,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
67,Central Toronto,0.0,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
75,Downtown Toronto,0.0,Park,Playground,Trail,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


# Cluster 2

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,1.0,Pizza Place,Coffee Shop,Pool,Pharmacy,Gym,Skating Rink,Pub,Sandwich Place,Department Store,Dessert Shop
1,Downtown Toronto,1.0,Café,Restaurant,Bookstore,Japanese Restaurant,Bar,Bakery,Chinese Restaurant,French Restaurant,Italian Restaurant,Beer Bar
2,Etobicoke,1.0,Pharmacy,Fried Chicken Joint,Seafood Restaurant,Fast Food Restaurant,Restaurant,Café,Mexican Restaurant,Hobby Shop,Bakery,Liquor Store
3,North York,1.0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,Downtown Toronto,1.0,Gay Bar,Ramen Restaurant,Bookstore,Smoke Shop,Ethiopian Restaurant,Bubble Tea Shop,Burger Joint,Salon / Barbershop,Burrito Place,Juice Bar
5,Scarborough,1.0,Motel,American Restaurant,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
10,Etobicoke,1.0,Grocery Store,Bakery,Convenience Store,Discount Store,Burrito Place,Burger Joint,Sandwich Place,Fast Food Restaurant,Supplement Shop,Tanning Salon
11,West Toronto,1.0,Bakery,Pharmacy,Grocery Store,Music Venue,Middle Eastern Restaurant,Café,Brewery,Supermarket,Bar,Bank
12,North York,1.0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Dim Sum Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
13,Downtown Toronto,1.0,Café,Bar,Dumpling Restaurant,Mexican Restaurant,Cocktail Bar,Beer Bar,Belgian Restaurant,Caribbean Restaurant,Cheese Shop,Grocery Store


# Cluster 3

In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,York,2.0,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Restaurant,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
43,Scarborough,2.0,Fast Food Restaurant,Wings Joint,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
54,North York,2.0,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


# Cluster 4

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Etobicoke,3.0,Baseball Field,Yoga Studio,Fast Food Restaurant,Falafel Restaurant,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


# Cluster 5

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Etobicoke,4.0,Home Service,Yoga Studio,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
